In [144]:
# Q1:
import torchvision
import torch
import torch.nn as nn
from torchvision import transforms, datasets


mnist_train=torchvision.datasets.MNIST('mnist',
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ]))

mnist_test=torchvision.datasets.MNIST('mnist',
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ]))

# train = torchvision.datasets.MNIST('', train=True, download=True,
#                        transform=transforms.Compose([
#                            transforms.ToTensor()
#                        ]))

# test = torchvision.datasets.MNIST('', train=False, download=True,
#                        transform=transforms.Compose([
#                            transforms.ToTensor()
#                        ]))

print(type(mnist_train))


<class 'torchvision.datasets.mnist.MNIST'>


In [145]:
print(mnist_train.data.shape)
print(mnist_test.data.shape)
# mnist_train.data.reshape((60000, 784)).shape

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


In [149]:
# Dataloader for laoding in data
# Steps to do this is to convert raw data to tensor
# Then tensor to dataset object
# And then from dataset object to dataloader

# Our MNIST is already in tensorDataset format
# train_dataset = torch.utils.data.TensorDataset(mnist_train.data, mnist_train.targets)
trainset = torch.utils.data.DataLoader(mnist_train, batch_size=264, shuffle=True)

# Same for test dataset
testset = torch.utils.data.DataLoader(mnist_test, batch_size=264, shuffle=False)



In [150]:
class NN(nn.Module):
    
    def __init__(self):
        super(NN, self).__init__() # any init calls before this should also call this init only
        self.fp_input = nn.Linear(28*28, 1024)
        self.fp1 = nn.Linear(1024, 1024)
        self.fp2 = nn.Linear(1024, 1024)
        self.fp3 = nn.Linear(1024, 1024)
        self.fp4 = nn.Linear(1024, 1024)
        self.fp5 = nn.Linear(1024, 10)
        self.output_activation = nn.Softmax(dim = 1)
        
    def forward(self, input):
        # flatten input already so that it is a row vector
        input = self.fp_input(input)
        input = self.fp1(input)
        input = self.fp2(input)
        input = self.fp3(input)
        input = self.fp4(input)
        input = self.output_activation(self.fp5(input))        
        return input        
    

In [154]:
# A minor difference is that the implementation of CrossEntrypyLoss implicitly applies a softmax activation followed by a log transformation 
# but NLLLoss does not.
torch.cuda.is_available()
device = torch.device('cuda')

In [159]:
# Calling the NN class
net = NN().to(device)

# Need to set optimizers and loss functions
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)


epochs = 10
for epoch in range(1):
    for i, (data, labels) in enumerate(trainset):
        optimizer.zero_grad()
        output = net(data.reshape(-1, 784).to(device))
        loss = loss_func(output, labels)
        loss.backward()
        optimizer.step()
    print(loss)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument target in method wrapper_nll_loss_forward)